# Lab2 Report

ผู้จัดทำได้ทำการสร้าง model ทางกลจำลองของ Serial manipulator 3 DOF โดยทั้ง 3 DOF เป็น revolute joint โดยได้สร้างแบบจำลองเพื่อนำไปแสดงผลใน rviz และหา forward kinematics และ inverse kinematic เพื่อนำไปใช้ในการระบุตำแหน่ง end effector ของหุ่นยนต์ โดย kinematics ของ Serial manipulator เป็นตรามรูปด้านร่าง

## Kinematics Modeling of Serial Manipulator

<center><img src="pic/robot_kinematic.png" width="500"></center>

โดยสามรถสร้างตาราง DH ของหุ่นยนต์ ได้ดังนี้

| i-1 | i | a | alpha | d | theta
| --- | --- | --- | --- | --- | --- |
| 0 | 1 | 0 | 0 | 150 | $$\dfrac{\pi}{2}$$ |
| 1 | 2 | 0 | $$\dfrac{\pi}{2}$$  | 130 | 0 |
| 2 | 3 | 390 | $$-\pi$$ | 130 | 0 |

จากนั้นทำการเขียน kinematics server เพื่อ publish ค่า jointState ไปให้กับ robot_state_publisher และสร้าง service 3 ตัว ได้แก่ 1.set_joint ทำหน้าที่ในการตั้งค่า joint configuration 2.get_pose ทำหน้าที่คืนค่าตำแหน่งของ end effector เทียบกับ base_link 3.solve_ik ทำหน้าที่ในการหา inverse kinematic จาก task space ไปเป็น joint configuration และกำหนด joint limit ของแต่ละ joint

<center><img src="pic/init.png" width="1000"></center>

callback function เมื่อมีการใช้ service set_joint โดยเครื่องหมายลบหน้า self.jointstate.position[1] เพื่อให้ทิศหมุนตรงกับ kinematic drawing

<center><img src="pic/set_joint_callback.png" width="1000"></center>

callback function เมื่อมีการใช้ service get_pose

<center><img src="pic/get_pose_callback.png" width="1000"></center>

## Forward Kinematics

จากนั้นจึงสร้าง method ในการหาตำแหน่งของจุดจุดปลายโดยใช้ forward kinematics โดย q คือ joint configuration 

<center><img src="pic/fk.png" width="1000"></center>

จากนั้นจึงสร้าง launch file เพื่อใช้ในการนำ CAD ของ Serial manipulator ไปจำลองใน rviz

<center><img src="pic/cad_rviz.png" width="1500"></center>

ทดลองใช้ set_joint โดย service มีโครงสร้าง

sensor_msgs/JointState jointstate<br>
---<br>
geometry_msgs/Point taskspace<br>

<center><img src="pic/set_joint.png" width="1500"></center>

ทดลองใช้ get_pose โดย service มีโครงสร้าง<br>

---<br>
geometry_msgs/Point taskspace

<center><img src="pic/get_pose.png" width="1500"></center>

## Inverse Kinematics

### Geometry analysis

ในส่วนนี้ผู้จัดทำได้ทำการหา inverse kinematics โดยใช้วิธี geometry analysis เพื่อให้แปลง task space ที่ end effector มาเป็น joint configuration โดยจะหาได้จากความสัมพันธ์<br>
$$
q_{1} = atan2(y,x) - \dfrac{\pi}{2}
$$
$$
q_{2} = atan2(s_{2},c_{2}) โดยที่ c_{2} = \dfrac{x^{2}+y^{2}+z^{2}-l1^{2}-l2^{2}}{2*l1*l2}, s_{2} = {\gamma}*\sqrt{1-c2^{2}}
$$
$$
q_{3} = atan2(z-150,\sqrt{x^{2}+y^{2}}) - atan2(l2*s2,l1+l2*c2)
$$
โดยในส่วนของ $q_{1}$ จะเป็นส่วนที่หมุนฐานของหุ่นยนต์ในแนวระนาบขนานกับพื้น $q_{2}$และ$q_{3}$ จะเป็นส่วนที่หมุนแขนของหุ่นยนต์ตั้งฉากกับพื้น<br>
ในส่วนของสมการ $q_{1}$ ต้องมีการลบด้วย $\dfrac{\pi}{2}$ เพราะที่ home configuration แขนหุ่นจะชี้ไปทางทิศ +y เทียบกับ base_link และ $q_{3}$ ในส่วนของ z ต้องลบด้วย 150 เพราะ joint อยู่เหนือจากพื้น 150 มิลลิเมตร และตรวจสอบว่าจุดที่ end effector ไปจะเกินระยะของแขนหนุ่หรือไม่ โดยคิดเทียบจาก joint ที่เชื่อต่อ link_1 และ link_2

<center><img src="pic/ik.png" width="1000"></center>

ทดลองใช้ solve_ik โดย service มีโครงสร้าง<br>

geometry_msgs/Point taskspace<br>
std_msgs/Bool elbowup<br>
---<br>
std_msgs/Bool status<br>

ซึ่งเราจะกำหนด taskspace ซึ่งเป็นจุดปลายของ gripper และเลือกได้ส่าจะเป็น elbow-up หรือ elbow-down และถ้าสามารถหา inverse kinematics ได้ก็จะคืนค่า true กลับมา

<center><img src="pic/solve_ik_elbowup.png" width="1500"></center>

inverse kinematics แบบ elbow-up

<center><img src="pic/solve_ik_elbowdown.png" width="1500"></center>

inverse kinematics แบบ elbow-down

### Scipy minimize

การทดสอบหา inverse kinematics โดบใช้ scipy minimize ในการหาระยะการเคลื่อนที่ของ joint ที่น้อยที่สุด เพื่อให้ปลาย end effector ไปอยู่ในจุดที่ต้องการ โดยกำหนดระยะเชิงมุมต่ำสุดและสูงสุดของแต่ละ joint

<center><img src="pic/boundary.png" width="1000"></center>

จากนั้นกำหนด constrain ให้กับ joint ของ หุ่นยนต์ โดยมีความสัมพันธ์<br>
$$
g(q) = 0
$$
$$
h(q) \leq 0
$$
โดยที่ g คือ function forward kinematic ลบด้วย taskspace ที่กำหนดจาก service<br>
     h คือ function ที่กำหนดไม่ให้ joint configuration ไม่เกิน limit ที่ตั้งไว้

จากนั้นกำหนด cost function ที่จะใช้เป็นตัวบอกว่าตำแหน่งที่หามีความใกล้หรือไกลจากเป้าหมายเท่าไรซึ่งเราจะต้องหา ค่า joint configuration ให้ cost function มีค่าน้อยที่สุด

<center><img src="pic/cost_func.png" width="1000"></center>

<center><img src="pic/pose_xyz.png" width="1000"></center>

จากนั้นปรับ function inverse kinematic ให้ใช้ minimize และในส่วน inequality equation นั้นมี bug ทำให้ไม่สามารถหา optimal solution ของ joint configuration ได้ จึงใช้ boundary แทน

<center><img src="pic/minimize_ik.png" width="1000"></center>

ในกรณีที่ Serial manipulator ไม่สามารถไปยัง taskspace ที่กำหนดได้ก็จะคืนค่า false กลับมา

<center><img src="pic/taskspace_limit.png" width="1000"></center>

หลังจากทดสอบ solve_ik พบว่ามีบางกรณีที่ solve_ik มีปัญหา ไม่สามารถหา optimal solution ของ joint configuration ได้เช่น ในกรณีจาก home configuration ไปที่ x:600 y:200 z:0 ไม่ได้ แต่ต้องไปที่ x:600 y:200 z:0 ก่อน แล้วถึงจะไป x:600 y:200 z:0 ได้ เนื่องจาก initial joint configuration ทำให้ตัว solver ไม่สามารถหา global minimum ได้